In [7]:
from numpy_operation import *

In [8]:
import pandas as pd 
import numpy as np

from math import ceil
from tqdm import tqdm 

np.random.seed(0)

df = pd.read_csv("/Users/kimwoojin/cryptolab/HEaaN-SDK/heaan_sdk/ml/linear_model/linear_model/SNUCryptoLab/titanic.csv")
df2 = df[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Survived"]]
df2 = df2.dropna(axis=0)
df2.info()

def male(x):
    if x == "male":
        return 1
    else :
        return 0
df2["Sex"] = df2["Sex"].apply(male)

y = df2["Survived"].to_numpy()
X = df2[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]].to_numpy()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=0)

from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_test_scaled = std.transform(X_test)

x_df = pd.DataFrame(X_train_scaled, columns=["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"])
y_df = pd.DataFrame(y_train, columns=["target"])
train_data = pd.concat([x_df, y_df],axis=1)
x_df = pd.DataFrame(X_test_scaled, columns=["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"])
y_df = pd.DataFrame(y_test, columns=["target"])
test_data = x_df.copy()
# test_data = pd.concat([x_df, y_df],axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    714 non-null    int64  
 1   Sex       714 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     714 non-null    int64  
 4   Parch     714 non-null    int64  
 5   Fare      714 non-null    float64
 6   Survived  714 non-null    int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 44.6+ KB


In [9]:
p1 = Params(input_dim=6, output_dim=4)
p2 = Params(input_dim=4, output_dim=1)
s = Sigmoid()
g = GeLU()
layers = [p1, g, p2, s]
network = MLP(layers=layers, lr=1, num_epoch=2, batch_size=1024)

In [10]:
network.fit(train_data)

Epoch 2: 100%|██████████| 286/286 [00:00<00:00, 13533.37it/s]


In [11]:
y_prob = network.predict(test_data)

ans = []
for i in y_prob :
    if i > 0.5 :
        ans.append(1)
    else:
        ans.append(0)
acc = 0
for i in range(len(ans)):
    if ans[i] == y_test[i]:
        acc += 1
acc/len(y_test)

0.5524475524475524

In [81]:
import torch 
import torch.nn as nn 

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_size):
        super().__init__()
        self.dev = 'cpu'
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.output_dim = output_dim 
        self.model = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_size, bias=False, device=self.dev),
            nn.GELU(),
            nn.Linear(self.hidden_size, self.output_dim, bias=False, device=self.dev),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        y = self.model.forward(x)
        return y 

torch.manual_seed(0)

cel = nn.CrossEntropyLoss()
# cel = nn.BCEWithLogitsLoss()

model = MLP(6, 1, 4)
model.to('cpu')
optimizer = torch.optim.SGD(model.parameters(), lr=1)

num_epoch = 3
from tqdm import tqdm 
for epoch in tqdm(range(num_epoch)) :
    model.train()
    optimizer.zero_grad()
    y = model(torch.tensor(X_train_scaled, dtype=torch.float32))
    target = torch.tensor(y_train, dtype=torch.float32)
    print(target)
    loss = cel(y, target.view(-1, 1))
    loss.backward()
    optimizer.step()
    print(loss)


100%|██████████| 3/3 [00:00<00:00, 223.15it/s]

tensor([1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
        1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
        0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
        1., 0., 0., 1., 0., 0., 0., 0., 

In [82]:
y_prob = model(torch.tensor(X_test_scaled, dtype=torch.float32))
# y_prob = model(torch.tensor(X_train_scaled, dtype=torch.float32))

In [83]:
l = []
acc = 0
for i in y_prob[:, 0]:
    
    if i < 0.5:
        l.append(0)
    else:
        l.append(1)
for i in range(len(l)):
    if l[i] == y_test[i]:
        acc += 1


In [84]:
acc /len(y_test)

0.7202797202797203